In [178]:
from matplotlib.axis import YAxis
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import math
import json

In [179]:
#Import rib.gg Data
df = pd.read_csv(r"C:\Users\Renzjordan\OneDrive\MiniProj\VALImpact\data\VCT-NA-2022-Stage-2-Challengers-ImpactEssentialPlayerData.csv")

In [180]:
#Set Attack Team Win target variable
df['ATKWin'] = 0
for i in range(0, len(df)):

    if(df.loc[i, 'attackingTeamNumber'] == df.loc[i, 'winningTeamNumber']):
        df.loc[i, 'ATKWin'] = 1

In [181]:
#Seperate Independent and Dependent Variables
indVars = df.drop(columns=['roundId_x', 'attackingTeamNumber', 'winningTeamNumber','ATKWin', 'playerId', 'assistants', 'victimId'])
depVar = df[['matchId_y', 'ATKWin']]

In [182]:
#Train on group stage
X_train = indVars[indVars['matchId_y'] < 69969].drop(columns=['matchId_y'])
Y_train = depVar[depVar['matchId_y'] < 69969].drop(columns=['matchId_y'])

X_test = indVars[indVars['matchId_y'] >= 69969].drop(columns=['matchId_y'])
X_test_MatchID = indVars[indVars['matchId_y'] >= 69969].drop_duplicates(subset=['matchId_y'])
y_test = depVar[depVar['matchId_y'] >= 69969].drop(columns=['matchId_y'])

In [183]:
#do Logisistic Regression
logreg = LogisticRegression()

logreg.fit(X_train, Y_train.values.ravel())

LogisticRegression()

In [184]:
#Get probability of [0, 1]
y_pred=logreg.predict_proba(X_test)
print(len(y_pred))

6827


In [185]:
#Transform Bomb Time from (100000 to -450000) -> (0 to 1)
def NormalizeData(data):
    # return (data - np.max(data)) / (np.min(data) - np.max(data))
    return (data - np.max(data)) / (-45000 - np.max(data))

In [186]:
#get list of halftimes
halftimes = {}
# print(len(X_test))

for i in X_test_MatchID.matchId_y:
    # print(i)
    #Set X-axis for graph (round + event time)
    xAx = df[df['matchId_y'] == i]['roundId_x']-df[df['matchId_y'] == i]['roundId_x'][df[df['matchId_y'] == i].index[0]] +1 + NormalizeData(df[df['matchId_y'] == i]['roundTime'])
    

    #Find index of round 13
    halftime = xAx.loc[xAx==13].index[0] 
    firstRound = xAx.loc[xAx==1].index[0] 
    lastRound = xAx.last_valid_index()

    halftimes[i] = [firstRound, halftime, lastRound]



print(halftimes)
# print(df.iloc[2354:2366])



{71394: [0, 115, 210], 71393: [211, 315, 380], 71392: [381, 493, 531], 71391: [532, 635, 766], 71399: [767, 854, 895], 71398: [896, 1008, 1176], 71397: [1177, 1285, 1379], 71396: [1380, 1484, 1581], 70727: [1582, 1694, 1783], 70726: [1784, 1888, 1966], 70725: [1967, 2082, 2190], 70723: [2191, 2301, 2357], 70722: [2358, 2452, 2553], 70721: [2554, 2659, 2766], 70720: [2767, 2879, 2950], 70629: [2951, 3055, 3168], 70628: [3169, 3273, 3371], 70627: [3372, 3466, 3581], 70626: [3582, 3705, 3809], 70625: [3810, 3910, 3992], 70624: [3993, 4098, 4135], 69979: [4136, 4240, 4279], 69978: [4280, 4380, 4438], 69976: [4439, 4547, 4594], 69975: [4595, 4710, 4813], 69986: [4814, 4928, 5038], 69985: [5039, 5151, 5200], 69984: [5201, 5312, 5406], 69982: [5407, 5520, 5637], 69981: [5638, 5758, 5818], 69973: [5819, 5935, 6020], 69972: [6021, 6124, 6205], 69971: [6206, 6319, 6429], 69970: [6430, 6549, 6619], 69969: [6620, 6730, 6826]}


In [187]:
#Get probabilities for home team
yAx = []
for i in X_test_MatchID.matchId_y:

    firstRound = halftimes[i][0]
    halftime = halftimes[i][1]
    lastRound = halftimes[i][2]

    if(df[df['matchId_y'] == i].iloc[0]['attackingTeamNumber'] == 1):
        firstHalf = [item[1] for item in y_pred[firstRound:halftime]]
        secHalf = [item[0] for item in y_pred[halftime:lastRound+1]]
    else:
        firstHalf = [item[0] for item in y_pred[firstRound:halftime]]
        secHalf = [item[1] for item in y_pred[halftime:lastRound+1]]
    #Sey y-axis for graph
    full = firstHalf + secHalf
    yAx += full
# print(yAx[0:30])
print(len(yAx))


6827


In [188]:
#Import player data for each match
playerData = pd.read_csv(r"C:\Users\Renzjordan\OneDrive\MiniProj\VALImpact\data\VCT-NA-2022-Stage-2-Challengers-PlayerData.csv")

#create impact df
playerImpact = df[['matchId_y', 'roundId_x', 'attackingTeamNumber', 'playerId', 'assistants', 'victimId']]
playerImpact = playerImpact[playerImpact['matchId_y'] >= 69969]
playerImpact['impact'] = 0

print(len(playerImpact))


6827


In [189]:
#Calculate impact for each player
for i in range(0, len(playerImpact)):
    
    matchId = playerImpact.loc[playerImpact.index[0]+i, 'matchId_y']
    playerId = playerImpact.loc[playerImpact.index[0]+i, 'playerId'] 

    if(not math.isnan(playerId)):
        # print(matchId, playerId)
        # print(playerData[((playerData['matchId'] == matchId) & (playerData['playerId'] == playerId))]['teamNumber'].values)
    
        playerImpact.loc[playerImpact.index[0]+i, 'impact'] = abs(yAx[i]-yAx[i-1])
    
    else:
        pass

In [190]:
#Get player info
playerNames = pd.read_csv(r"C:\Users\Renzjordan\OneDrive\MiniProj\VALImpact\data\VCTPlayers.csv")

In [191]:
#gVR = Victim impact by round, gVM = Victim impact by playoffs
gVR = playerImpact.groupby(['roundId_x', 'victimId'])['impact'].sum().reset_index()
gVM = playerImpact.groupby(['victimId'])['impact'].sum().reset_index()

print(gVM.head(10))

   victimId     impact
0     122.0  14.252229
1     165.0  18.544911
2     269.0  16.545354
3     369.0  37.754441
4     398.0  18.657079
5     494.0  25.346517
6     763.0  16.992538
7     789.0  32.282857
8     905.0   6.416316
9    1012.0  15.507543


In [192]:
#Getting impact for assistants + split individual impact
playerImpact['impact_assist'] = 0
for i in range(0, len(playerImpact)):
    if(len(json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])) > 0):
        playerImpact.loc[playerImpact.index[0]+i,'impact_assist'] = playerImpact.loc[playerImpact.index[0] + i, 'impact'] * (0.2 / len(json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])))
        playerImpact.loc[playerImpact.index[0]+i,'impact'] -= playerImpact.loc[playerImpact.index[0] + i, 'impact'] * (0.2)

In [193]:
#Create assistant impact dataframe
ass = pd.DataFrame(pd.np.empty((0, 8)))
ass.columns = playerImpact.columns.tolist()

for i in range(0, len(playerImpact)):
    for p in (json.loads(playerImpact.loc[playerImpact.index[0] + i, 'assistants'])):
        # print(p)
        row =  pd.DataFrame([{'matchId_y': playerImpact.loc[playerImpact.index[0]+i,'matchId_y'],
         'roundId_x': playerImpact.loc[playerImpact.index[0]+i,'roundId_x'], 
         'attackingTeamNumber': playerImpact.loc[playerImpact.index[0]+i,'attackingTeamNumber'],
         'playerId': p,
         'assistants': [],
         'victimId': playerImpact.loc[playerImpact.index[0]+i,'victimId'],
         'impact': playerImpact.loc[playerImpact.index[0]+i,'impact_assist']}])
        ass = pd.concat([ass, row])
        print(row)

ass['impact_assist'] = 0


C:\Users\Renzjordan\AppData\Local\Temp\ipykernel_2900\720036083.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  ass = pd.DataFrame(pd.np.empty((0, 8)))


   matchId_y  roundId_x  attackingTeamNumber  playerId assistants  victimId  \
0      71394    1113903                    2      3346         []    2716.0   

     impact  
0  0.007857  
   matchId_y  roundId_x  attackingTeamNumber  playerId assistants  victimId  \
0      71394    1113903                    2      2841         []    3582.0   

     impact  
0  0.002918  
   matchId_y  roundId_x  attackingTeamNumber  playerId assistants  victimId  \
0      71394    1113904                    2      2841         []    3346.0   

     impact  
0  0.000582  
   matchId_y  roundId_x  attackingTeamNumber  playerId assistants  victimId  \
0      71394    1113904                    2      2483         []    1802.0   

     impact  
0  0.001197  
   matchId_y  roundId_x  attackingTeamNumber  playerId assistants  victimId  \
0      71394    1113905                    2       494         []     398.0   

     impact  
0  0.013234  
   matchId_y  roundId_x  attackingTeamNumber  playerId assistants

In [194]:
#Impact by playoff

gA = ass.groupby(['playerId'])['impact'].sum().reset_index()

gP = playerImpact.groupby(['playerId'])['impact'].sum().reset_index()

# print(gP.head(20))
#print(gVM.head(20))
#print(gP.head(20))
#print(gA.head(20))

gP['impact'] = gP['impact'] + gA['impact'] - gVM['impact']

gP = pd.merge(gP, playerNames, how='left', left_on = 'playerId', right_on = 'Player Id')



print(gP.sort_values('impact', ascending=False))

    playerId     impact  Player Id        Ign               Team
7      789.0  12.846832        789      dicey          FaZe Clan
22    1994.0   8.897994       1994    supamen          FaZe Clan
20    1802.0   5.369068       1802  Cryocells               XSET
26    2716.0   4.918918       2716        yay       OpTic Gaming
34    3346.0   4.046951       3346        BcJ               XSET
28    2882.0   3.945804       2882        C0M      Evil Geniuses
6      763.0   3.605236        763      Asuna        100 Thieves
30    3062.0   3.275776       3062      AYRIN               XSET
9     1012.0   2.825921       1012       bang        100 Thieves
0      122.0   2.578203        122    stellar        100 Thieves
24    2483.0   2.514637       2483   crashies       OpTic Gaming
17    1763.0   2.504577       1763        tex        NRG Esports
15    1597.0   1.737856       1597       eeiu        NRG Esports
35    3582.0   1.667444       3582     zekken               XSET
14    1578.0   1.418439  